In [1]:
import numpy as np
import pandas as pd
import os

root_path = '/Users/shijunshen/Documents/Code/dataset/RL/AutonomousGreenhouseChallenge_edition2/Reference'

# Condition

In [2]:
greenhouse_path = os.path.join(root_path, 'GreenhouseClimate.csv')

greenhouse_df = pd.read_csv(greenhouse_path, low_memory=False)
# Tair（温度）, Rhair（湿度）, CO2air, EC_drain_PC, pH_drain_PC, water_sup
greenhouse_df = greenhouse_df[['%time', 'Tair', 'Rhair', 'CO2air', 'EC_drain_PC', 'pH_drain_PC', 'water_sup']]

greenhouse_df.head()

,%time,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43815.00000,20.9,60.6,472,4.0,6.3,4.0
1,43815.00347,20.7,61.4,501,4.0,6.3,6.0
2,43815.00694,21.2,60.9,489,4.0,6.3,6.0
3,43815.01042,20.7,62.5,497,4.0,6.3,6.0
4,43815.01389,20.7,62.7,477,4.0,6.3,6.0


In [3]:
# get Avg value for each day
greenhouse_df = greenhouse_df.apply(pd.to_numeric, errors='coerce')

greenhouse_dict = {}
temp_dict = {}

for column in greenhouse_df.columns:
    greenhouse_dict[column] = []
    temp_dict[column] = []

current_day = int(greenhouse_df['%time'][0])
temp_df = pd.DataFrame(columns=greenhouse_df.columns)
greenhouse_avgday_df = pd.DataFrame(columns=greenhouse_df.columns)
water_sup = 0
for index, row in greenhouse_df.iterrows():
    if int(row['%time']) == current_day:
        temp_df = pd.concat([temp_df, row.to_frame().T])
        if water_sup < row['water_sup']:
            water_sup = row['water_sup']
    else:
        temp_df = temp_df.mean(axis=0)
        temp_df['%time'] = current_day
        temp_df['water_sup'] = water_sup
        greenhouse_avgday_df = pd.concat([greenhouse_avgday_df, temp_df.to_frame().T])
        temp_df = pd.DataFrame(columns=greenhouse_df.columns)
        current_day = int(row['%time'])
        water_sup = 0
greenhouse_avgday_df

,%time,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43815.0,19.747917,66.430208,562.902778,3.977083,6.297222,6.0
0,43816.0,17.690941,76.064808,566.564460,4.005575,6.305923,2.0
0,43817.0,17.634843,72.871429,624.749129,4.000000,6.300000,0.0
0,43818.0,17.860279,75.823693,612.045296,4.001045,6.303833,2.0
0,43819.0,17.824042,73.118467,608.574913,3.920906,6.413240,3.0
...,...,...,...,...,...,...,...
0,43976.0,23.643554,71.581185,562.449477,10.160627,4.343206,38.0
0,43977.0,24.359930,67.695470,544.452962,10.222300,4.302091,26.0
0,43978.0,24.316725,68.102439,536.912892,10.155749,4.320209,25.0
0,43979.0,24.561324,62.458885,534.174216,10.123345,4.355401,26.0


In [4]:
# Data cleaning (set Nan value to the previous value)
greenhouse_avgday_df = greenhouse_avgday_df.apply(pd.to_numeric, errors='coerce')
greenhouse_avgday_df.fillna(method='ffill', inplace=True)
greenhouse_avgday_df.fillna(method='bfill', inplace=True)
greenhouse_avgday_df

,%time,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43815.0,19.747917,66.430208,562.902778,3.977083,6.297222,6.0
0,43816.0,17.690941,76.064808,566.564460,4.005575,6.305923,2.0
0,43817.0,17.634843,72.871429,624.749129,4.000000,6.300000,0.0
0,43818.0,17.860279,75.823693,612.045296,4.001045,6.303833,2.0
0,43819.0,17.824042,73.118467,608.574913,3.920906,6.413240,3.0
...,...,...,...,...,...,...,...
0,43976.0,23.643554,71.581185,562.449477,10.160627,4.343206,38.0
0,43977.0,24.359930,67.695470,544.452962,10.222300,4.302091,26.0
0,43978.0,24.316725,68.102439,536.912892,10.155749,4.320209,25.0
0,43979.0,24.561324,62.458885,534.174216,10.123345,4.355401,26.0


# Result

In [5]:
# quality

quality_path = os.path.join(root_path, 'TomQuality.csv')

quality_df = pd.read_csv(quality_path, low_memory=False)

quality_df = quality_df.apply(pd.to_numeric, errors='coerce')
quality_df.fillna(method='ffill', inplace=True)
quality_df.fillna(method='bfill', inplace=True)
quality_df

,%time,Flavour,TSS,Acid,%Juice,Bite,Weight,DMC_fruit
0,43880,74,7.9,15.0,58,187,7.77,9.59
1,43894,73,7.9,14.4,63,217,8.90,9.59
2,43908,76,8.7,15.8,55,284,7.47,9.59
3,43922,79,9.2,13.4,62,214,9.10,8.79
4,43936,73,8.6,12.1,58,238,10.10,8.74
5,43950,71,8.3,10.7,62,219,11.30,8.14
6,43964,78,9.0,10.9,67,185,14.63,9.32
7,43980,73,8.1,11.0,59,166,13.20,9.47


# Combine Result and Condition

In [6]:
# set the first timestamp of result as the beginning in both condition and result
temp_sum = pd.DataFrame(columns=greenhouse_avgday_df.columns)
greenhouse_avg_interval_df = pd.DataFrame(columns=greenhouse_df.columns)

for index, row in quality_df.iterrows():
    # print(index, row)
    if index == 0:
        continue
    left_time = quality_df['%time'][index-1]
    right_time = quality_df['%time'][index]
    for i, r in greenhouse_avgday_df.iterrows():
        if r['%time'] < left_time:
            continue
        if r['%time'] >= right_time:
            temp_sum = temp_sum.mean(axis=0)
            temp_sum['%time'] = left_time
            greenhouse_avg_interval_df = pd.concat([greenhouse_avg_interval_df, temp_sum.to_frame().T])
            temp_sum = pd.DataFrame(columns=greenhouse_avgday_df.columns)
            # add last timestamp info
            if index == quality_df.shape[0] - 1 and r['%time'] == right_time:
                greenhouse_avg_interval_df = pd.concat([greenhouse_avg_interval_df, r.to_frame().T])
            break
        temp_sum = pd.concat([temp_sum, r.to_frame().T])
    
greenhouse_avg_interval_df

,%time,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43880.0,22.732207,87.072375,608.535396,5.772066,5.594460,66.357143
0,43894.0,22.707242,87.600249,670.625187,4.946690,5.352987,75.714286
0,43908.0,22.915349,84.702180,760.527094,5.828616,5.187611,70.928571
0,43922.0,23.690393,78.294002,736.831757,5.470458,5.308711,81.000000
0,43936.0,22.942758,78.266078,731.763813,6.604629,5.038576,84.785714
0,43950.0,22.552912,77.995072,653.256346,7.142409,4.178223,70.357143
0,43964.0,23.824652,72.685453,590.995645,8.961803,4.143619,48.062500
0,43980.0,24.022997,60.286063,513.944251,10.146690,4.434843,26.000000


In [9]:
# discretize action space
a_tair = np.arange(12, 30, 4)  # temperature degree
a_rhair = np.arange(10, 100, 10)  # humidity %
a_co2air = np.arange(100, 2500, 200)  # CO2 ppm
a_ec_drain = np.arange(0, 15, 2)  # EC dS/m
a_ph_drain = np.arange(3, 9, 1)  # pH
a_water_sup = np.arange(0, 300, 30)  # Cumulative number of minutes of irrigation in a day (minutes)

print(a_tair, '\n', a_rhair, '\n', a_co2air, '\n', a_ec_drain, '\n', a_ph_drain, '\n', a_water_sup)


def discretize_column(column, bins):
    return np.digitize(column, bins)

greenhouse_avg_interval_df['Tair'] = discretize_column(greenhouse_avg_interval_df['Tair'], a_tair)
greenhouse_avg_interval_df['Rhair'] = discretize_column(greenhouse_avg_interval_df['Rhair'], a_rhair)
greenhouse_avg_interval_df['CO2air'] = discretize_column(greenhouse_avg_interval_df['CO2air'], a_co2air)
greenhouse_avg_interval_df['EC_drain_PC'] = discretize_column(greenhouse_avg_interval_df['EC_drain_PC'], a_ec_drain)
greenhouse_avg_interval_df['pH_drain_PC'] = discretize_column(greenhouse_avg_interval_df['pH_drain_PC'], a_ph_drain)
greenhouse_avg_interval_df['water_sup'] = discretize_column(greenhouse_avg_interval_df['water_sup'], a_water_sup)

greenhouse_avg_interval_df


[12 16 20 24 28] 
 [10 20 30 40 50 60 70 80 90] 
 [ 100  300  500  700  900 1100 1300 1500 1700 1900 2100 2300] 
 [ 0  2  4  6  8 10 12 14] 
 [3 4 5 6 7 8] 
 [  0  30  60  90 120 150 180 210 240 270]


,%time,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43880.0,3,8,3,3,3,3
0,43894.0,3,8,3,3,3,3
0,43908.0,3,8,4,3,3,3
0,43922.0,3,7,4,3,3,3
0,43936.0,3,7,4,4,3,3
0,43950.0,3,7,3,4,2,3
0,43964.0,3,7,3,5,2,2
0,43980.0,4,6,3,6,2,1


In [10]:
merged_df = pd.merge(quality_df, greenhouse_avg_interval_df, on='%time', how='outer')
merged_df

,%time,Flavour,TSS,Acid,%Juice,Bite,Weight,DMC_fruit,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43880,74,7.9,15.0,58,187,7.77,9.59,3,8,3,3,3,3
1,43894,73,7.9,14.4,63,217,8.90,9.59,3,8,3,3,3,3
2,43908,76,8.7,15.8,55,284,7.47,9.59,3,8,4,3,3,3
3,43922,79,9.2,13.4,62,214,9.10,8.79,3,7,4,3,3,3
4,43936,73,8.6,12.1,58,238,10.10,8.74,3,7,4,4,3,3
5,43950,71,8.3,10.7,62,219,11.30,8.14,3,7,3,4,2,3
6,43964,78,9.0,10.9,67,185,14.63,9.32,3,7,3,5,2,2
7,43980,73,8.1,11.0,59,166,13.20,9.47,4,6,3,6,2,1
